# **Laboratorio 9**
**Daniela Navas**

## **Task 1** - Teoría

**Responda las siguientes preguntas de forma clara y concisa, pueden subir un PDF o bien dentro del mismo Jupyter Notebook.**<br> 
**1. Diga cual es la diferencia entre Modelos de Markov y Hidden Markov Models**<br>
En los **Modelos de Markov** los estados son directamente observables. La probabilidad de transición entre estados depende únicamente del estado actual, no de los estados anteriores. Se utilizan comúnmente en aplicaciones donde los estados son visibles, como la predicción del clima o la generación de texto. En cambio, en los **Hidden Markov Models (HMM)** los estados no son directamente observables (son "ocultos"). En cambio, se observan emisiones que dependen de estos estados ocultos. Los HMM son útiles en situaciones donde los estados subyacentes no pueden ser observados directamente, como en el reconocimiento de voz o la bioinformática.

**2. Investigue qué son los factorial HMM (Hidden Markov Models)**<br>
Los factorial HMM son una extensión de los HMM estándar. En lugar de tener una sola cadena de estados ocultos, los factorial HMM tienen múltiples cadenas de estados ocultos que interactúan entre sí. Esto permite modelar sistemas más complejos donde múltiples factores pueden influir en las observaciones. Se utilizan en aplicaciones como el análisis de datos multivariados y la modelación de sistemas biológicos.

**3. Especifique en sus propias palabras el algoritmo Forward Backward para HMM**<br>
El algoritmo Forward Backward es un método de inferencia para HMM que calcula las probabilidades posteriores de los estados ocultos dados una secuencia de observaciones. Se realiza en dos pasos:
1. **Forward:** Calcula las probabilidades de estar en cada estado en cada punto de tiempo, dado las observaciones hasta ese punto.
2. **Backward:** Calcula las probabilidades de observar las secuencias restantes desde cada punto de tiempo hacia adelante. Al combinar estas probabilidades, se obtiene la distribución de los estados en cualquier punto de tiempo dado toda la secuencia de observaciones.

**4. En el algoritmo de Forward Backward, por qué es necesario el paso de Backward (puede escribir ejemplos o casos para responder esta pregunta)**<br>
El paso de Backward es crucial porque permite incorporar información futura en el cálculo de las probabilidades de los estados actuales. Sin este paso, solo se tendría en cuenta la información pasada y presente, lo que podría llevar a estimaciones menos precisas. Por ejemplo, si estás modelando el reconocimiento de voz, la información sobre las palabras futuras puede ayudar a determinar mejor los estados actuales de los fonemas.

---

## **Task 2** - Algoritmo Forward Backward en HMM 
El algoritmo forward-backward se basa en una forma de programación dinámica. Para este task deberán investigar sobre el mismo. En este ejercicio estamos ante un modelo meteorológico representado por un Modelo Oculto de Markov (HMM) con dos estados: "Soleado" y "Lluvioso". Queremos predecir el tiempo en un día determinado basándonos en las observaciones de si el día anterior estuvo soleado o lluvioso.

Con esto, considere lo siguiente: 
1. Defina los parámetros del modelo oculto de Markov (HMM), incluidos estados, observaciones, probabilidades iniciales, probabilidades de transición y probabilidades de emisión. 
2. Cree una instancia de la clase HMM con los parámetros definidos. 
3. Genere una secuencia de observaciones utilizando el método generate_sequence. Este paso es opcional pero útil para realizar pruebas. 
4. Utilice el método forward para calcular las probabilidades directas, que representan la probabilidad de observar una secuencia particular de observaciones hasta un paso de tiempo específico y estar en un estado particular en ese paso de tiempo. 
5. Utilice el método backward para calcular las probabilidades hacia atrás, que representan la probabilidad de observar la secuencia restante desde un paso de tiempo específico hasta el final de la secuencia, dado que el sistema se encuentra en un estado particular en ese paso de tiempo. 
6. Utilice el método compute_state_probabilities para calcular las probabilidades de estar en cada estado en cada paso de tiempo dada la secuencia observada. Este paso implica combinar las probabilidades hacia adelante y hacia atrás. 
7. Imprima o analice las probabilidades calculadas según sea necesario. 

In [11]:
# ------------------------------------------------------- 
#
# Definición de la clase HMM (Hidden Markov Model)
# 
# -------------------------------------------------------
class HMM:
    def __init__(self, states, observations, initial_prob, transition_prob, emission_prob):
        """
        Constructor del modelo HMM.
        
        Params:
        - states: lista de estados ocultos posibles.
        - observations: lista de observaciones posibles.
        - initial_prob: diccionario con las probabilidades iniciales de cada estado.
        - transition_prob: diccionario de diccionarios que representa las probabilidades de transición entre estados.
        - emission_prob: diccionario de diccionarios que representa las probabilidades de emisión de observaciones dado un estado.
        """

        self.states = states
        self.observations = observations
        self.initial_prob = initial_prob
        self.transition_prob = transition_prob
        self.emission_prob = emission_prob

    def generate_sequence(self, length):
        """
        Genera una secuencia de observaciones de una longitud dada a partir del modelo HMM.

        Param:
        - length: longitud de la secuencia a generar.

        Return:
        - sequence: lista de observaciones generadas por el modelo.
        """

        import random # Librería para seleccionar aleatoriamente elementos
        sequence = [] # Lista para almacenar la secuencia de observaciones generadas

        # Se elige un estado inicial basado en las probabilidades iniciales
        current_state = random.choices(self.states, weights=[self.initial_prob[s] for s in self.states])[0]

        # Generación de la secuencia de observaciones
        for _ in range(length):
            # Se genera una observación basada en la distribución de emisión del estado actual
            observation = random.choices(self.observations, weights=[self.emission_prob[current_state][o] for o in self.observations])[0]
            sequence.append(observation) # Se agrega la observación a la secuencia
            current_state = random.choices(self.states, weights=[self.transition_prob[current_state][s] for s in self.states])[0] # Elegir el siguiente estado basado en las probabilidades de transición
        return sequence  # Retorna la secuencia de observaciones generada

    def forward(self, observations):
        """
        Calcula la matriz de probabilidades hacia adelante (alpha) para una secuencia de observaciones.

        Param:
        - observations: lista de observaciones sobre las que se evaluará el modelo.

        Return:
        - alpha: lista de diccionarios, donde cada uno contiene las probabilidades de cada estado 
                 en cada instante de tiempo dado lo observado hasta ese punto.
        """

        alpha = [{}] # Lista de diccionarios: alpha[t][state] representa la probabilidad de estar en 'state' en el tiempo t dado las observaciones hasta t
        
        # Inicialización en t = 0 con la primera observación
        for state in self.states:
            alpha[0][state] = self.initial_prob[state] * self.emission_prob[state][observations[0]]

        # Recursión para t > 0
        for t in range(1, len(observations)):
            alpha.append({})
            for curr_state in self.states:
                # Suma de las probabilidades desde todos los estados anteriores al estado actual
                alpha[t][curr_state] = sum(alpha[t - 1][prev_state] * 
                                           self.transition_prob[prev_state][curr_state] for prev_state in self.states) * \
                                           self.emission_prob[curr_state][observations[t]]
        return alpha # Devuelve la matriz de probabilidades hacia adelante

    def backward(self, observations):
        """
        Calcula la matriz de probabilidades hacia atrás (beta) para una secuencia de observaciones.

        Param:
        - observations: lista de observaciones sobre las que se evaluará el modelo.

        Return:
        - beta: lista de diccionarios, donde cada uno contiene las probabilidades de cada estado 
                en cada instante de tiempo dado lo que falta por observar desde ese punto.
        """

        beta = [{} for _ in range(len(observations))] # Inicialización de la matriz beta con diccionarios vacíos
        for state in self.states:
            beta[-1][state] = 1.0  # Inicializar al final

        # Inicializar en el último tiempo con 1.0 para todos los estados (es el punto de partida de la recursión hacia atrás)
        for t in reversed(range(len(observations) - 1)):
            for state in self.states:
                beta[t][state] = sum(
                    self.transition_prob[state][next_state] *                         # probabilidad de transición al siguiente estado 
                    self.emission_prob[next_state][observations[t + 1]] *             # probabilidad de emitir la siguiente observación
                    beta[t + 1][next_state]                                           # beta del siguiente estado
                    for next_state in self.states
                )
        return beta # Devuelve la matriz de probabilidades hacia atrás

    def compute_state_probabilities(self, observations):
        """
        Calcula la probabilidad de cada estado en cada instante de tiempo dado una secuencia de observaciones,
        utilizando los algoritmos forward y backward.

        Param:
        - observations: lista de observaciones para las cuales se calcularán las probabilidades de estado.

        Return:
        - state_probs: lista de diccionarios, donde cada diccionario representa las probabilidades 
                       normalizadas de estar en cada estado en el tiempo t dado toda la secuencia observada.
        """

        forward_probs = self.forward(observations)   # Calcular las probabilidades hacia adelante (alpha)
        backward_probs = self.backward(observations) # Calcular las probabilidades hacia atrás (beta)
        state_probs = []                             # Lista para almacenar las probabilidades por tiempo

        # Para cada instante de tiempo en la secuencia de observaciones
        for t in range(len(observations)):
            # Calcular la suma total (normalización) de las probabilidades forward * backward para todos los estados
            total = sum(forward_probs[t][s] * backward_probs[t][s] for s in self.states)
            
            # Calcular la probabilidad normalizada para cada estado en el tiempo t
            state_probs.append({
                s: (forward_probs[t][s] * backward_probs[t][s]) / total for s in self.states
            })
        return state_probs # Devuelve la lista de distribuciones de probabilidad por estado y tiempo

def imprimir_probabilidades(titulo, probs):
    """
    Imprime de forma formateada las probabilidades de los estados en cada paso de tiempo.

    Param:
    - titulo: título a mostrar antes de imprimir los resultados.
    - probs: lista de diccionarios, donde cada diccionario contiene las probabilidades 
             de los estados en un instante de tiempo (como el resultado de forward, backward o compute_state_probabilities).
    """
    print(f"\n{titulo}:") # Imprime el título como encabezado

    # Itera sobre la lista de probabilidades por tiempo
    for t, paso in enumerate(probs):
        print(f"Paso {t+1}:")
        for estado in paso:
            print(f"  {estado:<6}: {paso[estado]*100:.2f}%") # Imprimir cada estado y su probabilidad en porcentaje con dos decimales
        print("-" * 25)

# Parámetros del modelo (Definidos según el código brindado)
states = ['Sunny', 'Rainy']
observations = ['Sunny', 'Sunny', 'Rainy']
initial_prob = {'Sunny': 0.5, 'Rainy': 0.5}
transition_prob = {
    'Sunny': {'Sunny': 0.8, 'Rainy': 0.2},
    'Rainy': {'Sunny': 0.4, 'Rainy': 0.6}
}
emission_prob = {
    'Sunny': {'Sunny': 0.8, 'Rainy': 0.2},
    'Rainy': {'Sunny': 0.3, 'Rainy': 0.7}
}

# ==================================================================================================================
# Instancia del modelo
hmm = HMM(states, observations, initial_prob, transition_prob, emission_prob)

# 1. Generar secuencia de observaciones
obs_sequence = hmm.generate_sequence(5)

# 2. Calcular forward
forward_probs = hmm.forward(obs_sequence)

# 3. Calcular backward
backward_probs = hmm.backward(obs_sequence)

# 4. Calcular probabilidades de estado
state_probs = hmm.compute_state_probabilities(obs_sequence)

# --------------------------------
# Mostrar resultados 
print("\nSecuencia:")
print(" -> ".join(obs_sequence))

imprimir_probabilidades("Probabilidades Forward", forward_probs)
imprimir_probabilidades("Probabilidades Backward", backward_probs)
imprimir_probabilidades("Probabilidades de Estado (Forward-Backward)", state_probs)



Secuencia:
Rainy -> Rainy -> Sunny -> Sunny -> Rainy

Probabilidades Forward:
Paso 1:
  Sunny : 10.00%
  Rainy : 35.00%
-------------------------
Paso 2:
  Sunny : 4.40%
  Rainy : 16.10%
-------------------------
Paso 3:
  Sunny : 7.97%
  Rainy : 3.16%
-------------------------
Paso 4:
  Sunny : 6.11%
  Rainy : 1.05%
-------------------------
Paso 5:
  Sunny : 1.06%
  Rainy : 1.30%
-------------------------

Probabilidades Backward:
Paso 1:
  Sunny : 3.92%
  Rainy : 5.62%
-------------------------
Paso 2:
  Sunny : 15.32%
  Rainy : 10.45%
-------------------------
Paso 3:
  Sunny : 22.20%
  Rainy : 18.60%
-------------------------
Paso 4:
  Sunny : 30.00%
  Rainy : 50.00%
-------------------------
Paso 5:
  Sunny : 100.00%
  Rainy : 100.00%
-------------------------

Probabilidades de Estado (Forward-Backward):
Paso 1:
  Sunny : 16.61%
  Rainy : 83.39%
-------------------------
Paso 2:
  Sunny : 28.61%
  Rainy : 71.39%
-------------------------
Paso 3:
  Sunny : 75.05%
  Rainy : 24.95

*(Para este set de resultados específicos)*

El método **Forward** calcula las probabilidades de los estados ocultos considerando solo las observaciones anteriores. A medida que avanzan los pasos, se observa una disminución continua de las probabilidades, ya que el modelo solo tiene en cuenta la evidencia hasta el punto actual, sin considerar las observaciones futuras. Primero, la probabilidad de "Rainy" es bastante alta (35%), lo que refleja la primera observación (Rainy) y la transición probable a este estado. Sin embargo, las probabilidades de "Sunny" y "Rainy" son bajas en comparación, lo que indica incertidumbre debido a que no se tiene información de los pasos siguientes. A medida que avanzan los pasos, las probabilidades continúan disminuyendo, ya que el modelo no tiene acceso a las observaciones futuras. Esto lleva a una pérdida progresiva de certeza. Las probabilidades de "Sunny" y "Rainy" son bastante cercanas (1.06% y 1.30%), lo que refleja la alta incertidumbre al llegar al final de la secuencia.

El método **Backward** calcula las probabilidades de los estados ocultos utilizando solo la información de las observaciones futuras, es decir, comienza desde el último paso y retrocede. Este enfoque da una visión más completa de las probabilidades, ya que las observaciones futuras refuerzan la estimación de los estados actuales. A medida que se retrocede, las probabilidades de "Sunny" y "Rainy" aumentan considerablemente. El modelo, al tener acceso a la información futura, ajusta sus creencias de manera más precisa, reflejando una mayor certeza sobre el estado subyacente. Las probabilidades en el paso 5 (100% para ambos estados) no son exactas en un sentido práctico, ya que el modelo está utilizando información que ya ha sido observada, pero muestra cómo el método **Backward** se ajusta completamente a los estados observados.

En cambio, en **Forward-Backward** se combina lo mejor de ambos métodos, integrando la información de las observaciones pasadas y futuras. Esto da lugar a estimaciones más precisas de los estados ocultos, ya que se incorporan las evidencias completas para cada paso en la secuencia. El modelo predice una probabilidad bastante alta de "Rainy" (83.39%), ya que la información tanto pasada como futura fortalece esta estimación. Esto refleja la alta probabilidad de transición de "Rainy" a "Rainy" y la observación inicial.
Las probabilidades para "Sunny" y "Rainy" en estos pasos se equilibran más que en el enfoque **Forward**. El modelo ajusta sus estimaciones con mayor precisión, ya que ya tiene en cuenta las observaciones futuras y las transiciones pasadas.
En el último paso, se ve una ligera inclinación hacia "Rainy" (54.96%), lo que refleja cómo la secuencia de transiciones y las observaciones pasadas todavía influyen en el modelo, a pesar de la observación final de "Rainy".

---

**GITHUB:**
https://github.com/danielanavas2002/InteligenciaArtificial/tree/main/Laboratorio/Laboratorio09